In [38]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from model import CropDataset, CropYieldModel

In [39]:
# MAPE 계산 함수 추가
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## 평가함수

In [40]:
def evaluate_model(model, dataloader, device):
    model.eval()
    true_yields = []
    pred_yields = []
    
    with torch.no_grad():
        for images, features, labels in dataloader:
            images = images.to(device)
            features = features.to(device)
            labels = labels.to(device)
            
            outputs = model(images, features)
            pred_yields.extend(outputs.cpu().numpy().flatten())
            true_yields.extend(labels.cpu().numpy().flatten())
    
    true_yields = np.array(true_yields)
    pred_yields = np.array(pred_yields)
    
    mse = mean_squared_error(true_yields, pred_yields)
    rmse = np.sqrt(mse)
    r2 = r2_score(true_yields, pred_yields)
    mae = mean_absolute_error(true_yields, pred_yields)
    mape = mean_absolute_percentage_error(true_yields, pred_yields)
    
    return rmse, r2, mae, mape, true_yields, pred_yields


## 데이터 준비

In [41]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## 테스트 데이터셋
일차적으로는 원본데이터 활용했으나 추후 test 데이터 추가할 예정임.

In [42]:
test_dataset = CropDataset('data/json_directory/', 'data/images_directory/', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Creating dataset from data/json_directory/ and data/images_directory/
Found 50 json files
Loaded 50 valid samples out of 50 JSON files


In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CropYieldModel().to(device)
model.load_state_dict(torch.load('model/crop_yield_model.pth'))

<All keys matched successfully>

## 결과 

In [44]:
rmse, r2, mae, mape, true_yields, pred_yields = evaluate_model(model, test_dataloader, device)

print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R-squared Score: {r2:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Absolute Percentage Error: {mape:.2f}%")


Root Mean Squared Error: 30.4636
R-squared Score: 0.9209
Mean Absolute Error: 24.5909
Mean Absolute Percentage Error: 6.41%


In [45]:
plt.figure(figsize=(10, 6))
plt.scatter(true_yields, pred_yields, alpha=0.5)
plt.plot([min(true_yields), max(true_yields)], [min(true_yields), max(true_yields)], 'r--', lw=2)
plt.xlabel("True Yield")
plt.ylabel("Predicted Yield")
plt.title("True vs Predicted Yield")
plt.savefig("results/yield_prediction_scatter.png")
plt.close()